In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

base_url = 'http://notelections.online'

In [30]:
url = 'http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222'

In [33]:
data = pd.DataFrame(columns = ['ТИК','УИК'])
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
tiks = soup.find_all('a', style='text-decoration: none')
tiks.pop(-1)
soup = BeautifulSoup(requests.get('http://notelections.online' + tiks[0].get('href')).text, 'html.parser')
candidats = soup.find_all('table', bgcolor='#ffffff', cellpadding='2', cellspacing='1')[0].find_all('td', style='color:black', align='left', width='50')
cand = []
for candidat in candidats:
  cand.append(candidat.text)
candidats = pd.DataFrame(columns = cand)
data = pd.concat([data,candidats])


In [35]:
for tik in tiks:
  import pdb
  soup = BeautifulSoup(requests.get('http://notelections.online' + tik.get('href')).text, 'html.parser')
  find = soup.find_all('table', bgcolor='#ffffff', cellpadding='2', cellspacing='1')[-1]
  uiks = find.find_all('nobr')
  try:
    uik = len(find.find_all('tr', valign='top')[0].find_all('td'))
  except:
    pass
    #pdb.set_trace()
  for i in range(uik):
    main=[uiks[k].text for k in range(i, i + uik * 14 + 1, uik)]
    data.loc[len(data)] = [tik.text, *main]

In [37]:
data.to_csv('parsing.csv', index=False)